# Infinite Craft with AI

My son was telling me about a game where he could put things together and that would produce a new thing and it was infinite. So I thought this would be a good experiment with LLM.

1. Start with a list of 4 things - fire, water, air and earth
2. When any 2 things are put together, create another thing by following these rules :
    - Do a lookup in a database. If the combination of those two things already exists, then create the third thing per what is in the db.
    - If that combo does not yet exist, use a LLM to create the new thing and store in the database. Then add that new thing to the list of things that the person can craft with.

## Create the database

In [1]:
import duckdb

def addItem(name):
    conn = duckdb.connect("things.db")
    conn.sql("INSERT INTO things VALUES (nextval('seq_thingid'), '" + name +"')")

# create a connection to a file called 'file.db'
con = duckdb.connect("things.db")

# create tables
con.sql("CREATE TABLE things (Thingid INTEGER PRIMARY KEY, Name VARCHAR)")
con.sql("CREATE TABLE combos (combo VARCHAR PRIMARY KEY, Name VARCHAR)")
con.sql("CREATE SEQUENCE seq_thingid START 1")
con.close()

addItem('fire')
addItem('water')
addItem('air')
addItem('earth')

In [2]:
con = duckdb.connect("things.db")
con.sql("SELECT * FROM things").show()
con.close()

┌─────────┬─────────┐
│ Thingid │  Name   │
│  int32  │ varchar │
├─────────┼─────────┤
│       1 │ fire    │
│       2 │ water   │
│       3 │ air     │
│       4 │ earth   │
└─────────┴─────────┘



## Initialize the LLM

In [4]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

In [5]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [7]:
template = ("Things 1: {word1}, Thing 2: {word2} "
            "Answer: word association. Given those two things, Thing 1 and Thing 2, " 
            "here is the most likely Thing that I can think of with less than 3 words. "
            "Here is just that thing with no other text, no matter how odd that thing may " 
            "be in two words or less and no ending punctuation. It must be a plausible thing : ")

prompt = PromptTemplate.from_template(template)

In [8]:
llm = OpenAI()
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [10]:
llm_chain.invoke({"word1":"beans", "word2":"salsa"})

{'word1': 'beans', 'word2': 'salsa', 'text': '\ntaco salad'}

In [12]:
def craft(thing1, thing2):
    thing1 = thing1.lower()
    thing2 = thing2.lower()
    thing3 = ""

    con = duckdb.connect("things.db")
    con.execute("SELECT * FROM things where Name = '" + thing1 + "'")
    result1 = con.fetchone()
    con.execute("SELECT * FROM things where Name = '" + thing2 + "'")
    result2 = con.fetchone()

    if(result1 == None or result2 == None):
        print("your things must exist")
    else:
        con.execute("SELECT Name from combos WHERE combo = '" + (thing1 + thing2) + "'")
        result = con.fetchone();
        if(result == None):
            resp = llm_chain.invoke({"word1":thing1, "word2":thing2})
            thing3 = resp["text"].lower().strip().replace('.', '').replace("\"", "").replace("'","")
            print("you are first to craft: " + thing3)
            print("adding to inventory")

            con.sql("INSERT INTO things VALUES (nextval('seq_thingid'), '" + thing3 +"')")
            con.sql("INSERT INTO combos VALUES ('" + (thing1 + thing2) + "', '" + thing3 + "')")
            print("ineserted")
        else:
            thing3 = result[0]
            print(thing3)
    con.close()
    return thing3

In [60]:
craft("", "waterfall")

you are first to craft: natures wonder
adding to inventory
ineserted


In [61]:
con = duckdb.connect("things.db")
con.sql("SELECT * FROM things").show()
con.sql("SELECT * FROM combos").show()
con.close()

┌─────────┬─────────────────────┐
│ Thingid │        Name         │
│  int32  │       varchar       │
├─────────┼─────────────────────┤
│       1 │ fire                │
│       2 │ water               │
│       3 │ air                 │
│       4 │ earth               │
│       5 │ steam               │
│       6 │ heat source         │
│       7 │ hot springs         │
│       8 │ planet earth        │
│       9 │ habitat destruction │
│      10 │ heat wave           │
│       · │     ·               │
│       · │     ·               │
│       · │     ·               │
│      28 │ burning sensation   │
│      29 │ volcano             │
│      30 │ eruption            │
│      31 │ molten lava         │
│      32 │ volcanic explosion  │
│      33 │ volcano eruption    │
│      34 │ magma fountain      │
│      35 │ word association    │
│      36 │ waterfall           │
│      37 │ natures wonder      │
├─────────┴─────────────────────┤
│ 37 rows (20 shown)  2 columns │
└─────────────

In [63]:
import gradio as gr

def docraft(Thing1, Thing2):
    t = craft(Thing1, Thing2)
    return t

demo = gr.Interface(
    fn=docraft,
    inputs=["text", "text"],
    outputs=["text"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


steam
